# Stall prediction

 Prediction of the pressure profil of an aircraft wing based on deep learning.
 Our goal is to use pressure predictions to prevent aircraft stalls.
 The advantage of deep learning in this case is that it requires very little computation compared with real-time simulations, and delivers rapid results that can be used by pilots or drones in their piloting

We are using data coming from the deepstall project : https://projects.asl.ethz.ch/datasets/doku.php?id=deepstall

The data we have is in h5df, to visualize it, we can use : https://myhdf5.hdfgroup.org/

Important question : Continuous regression or classification ?

 ## Data Importation

We have 2 dataset : one in wind tunnel and another in real flight condition

In [75]:
import pandas as pd # Data Managing
import seaborn as sns # visual tool
import matplotlib.pyplot as plt # visual tool
import numpy as np

import h5py # to deal with h5p file



file_FD = h5py.File('Flight_Data/FD_cal.h5', "r")
file_WT = h5py.File('Wind_Tunnel/WT_cal.h5', "r")

FD_keys = list(file_FD.keys())
WT_keys = list(file_WT.keys())

FD_attribute =list(file_FD[FD_keys[0]].keys())
print(len(FD_attribute)) #548

WT_attribute =list(file_WT[WT_keys[0]].keys())
print(len(WT_attribute)) #506 

data_WT = {}
data_FD = {}
list_keys_WT = list(file_WT.keys())
list_keys_FD = list(file_FD.keys())


548
506


## Surface cleaning

In [74]:
#for attr in FD_attribute:
#    if attr not in WT_attribute:
#        print(attr) # not null

        
#for attr in WT_attribute:
#    if attr not in FD_attribute:
#        print(attr) # not null
        
    
# both data files are truncated to save only common fields
common_attr = []
for attr in WT_attribute:
    if attr in FD_attribute:
        common_attr.append(attr)
to_remove = []        
for attr in common_attr:
    if 'rc_' in attr:
        to_remove.append(attr)
    if 'battery' in attr:
        to_remove.append(attr)
    if 'cpu' in attr:
        to_remove.append(attr)
    if 'actuator' in attr:
        to_remove.append(attr)
    if 'manual' in attr:
        to_remove.append(attr)
        
for attr in to_remove:
        common_attr.remove(attr)


In [65]:
print("cleaned data column number : " + str(len(common_keys)) + " uncleaned :"  + str(len(FD_attribute)))

cleaned data column number : 370 uncleaned :548


### From h5d to dict and to pandas dataframe format

In [71]:

# for each key of each h5d file we create a dict which will be later transform into a pandas dataframe
dicts_list_WT = [{} for _ in range(len(list_keys_WT))]
dicts_list_FD = [{} for _ in range(len(list_keys_FD))]

# Populate the list of dict
for _ in range(len(list_keys_WT)):
    for attr in common_attr:
        dicts_list_WT[_][attr] = file_WT[list_keys_WT[_]][attr][0]
for _ in range(len(list_keys_FD)):
    for attr in common_attr:
        dicts_list_FD[_][attr] = file_FD[list_keys_FD[_]][attr][0]
    
global_dict_list = dicts_list_FD + dicts_list_WT


### From dicts to pandas data frame

an important decision is to be made here: are we going to mix all our data in one massive dataframe ? Or are we going to make small ones ?

For each keys (for exemple '11_aoa_sweep_m5') of the dataframe there is an environmental context, for exemple the experiment as realized in a wind tunnel or outside, during a foggy day or not, using this attack angle or this one. All those informations have an impact on the data, so if we mix datas we also mix contexts and we lose the **contextual information**.



In [73]:
# Then we transform each dict into a pandas dataframe and check if there is not any constant field
def rm_constant_field(df):
    """
    Drops constant value columns of pandas dataframe.
    """
    result = df.copy()
    for column in df.columns:
        if len(df[column].unique()) == 1:
            result = result.drop(column,axis=1)
    return result

df_list = [pd.DataFrame() for _ in range(len(dicts_list_WT) + len(dicts_list_FD))]
for _ in range(len(df_list)):
    df_list[_] = pd.DataFrame.from_dict(global_dict_list[_])
    #rm_constant_field(df_list[_]) might not be not a good idea



## Data preparation

## Draw